In [ ]:
from llama_index.core import SimpleDirectoryReader, Settings, VectorStoreIndex
from llama_index.readers.file.tabular.base import PandasCSVReader
import pandas as pd
import os
from InstructorEmbedding import INSTRUCTOR
from llama_index.core.embeddings import BaseEmbedding
from transformers import AutoTokenizer




In [ ]:
class InstructorEmbeddings(BaseEmbedding):
    """ 
        Creates an InstructorEmbeddings object that instantiates and INSTRUCTOR object model and uses it to encode documents and query for a specified domanin
    """

    def __init__(self, instructor_model, instruction, **kwargs):
        self._model = INSTRUCTOR(instructor_model)

        self._instruction = instruction
        super().__init__(**kwargs)

    def _get_query_embedding(self, query):
        embeddings = self._model.encode([[self._instruction, query]])

        return embeddings[0]

    def _get_text_embedding(self, text):
        embeddings = self._model.encode([[self._instruction, text]])
        return embeddings[0]

    def _get_text_embeddings(self, texts):
        embeddings = self._model.encode(
            [[self._instruction, text] for text in texts]
        )
        return embeddings

    async def _get_query_embedding(self, query):
        return self._get_query_embedding(query)

    async def _get_text_embedding(self, text):
        return self._get_text_embedding(text)
        

In [3]:
nba_articles = pd.read_csv('nba_espn_articles.csv')
nba_stats = pd.read_csv('nbastats_po_2022.csv')

In [ ]:
reader = PandasCSVReader(concat_rows = False, pandas_config={'usecols': ['content']})

documents = reader.load_data(file = 'nba_espn_articles.csv')



In [ ]:
Settings.embed_model = InstructorEmbeddings('hkulnp/intstructor-base', 'Represent the NBA game recap text for retrieval')

index = VectorStoreIndex(documents)

index.storage_context.persist('nba_articles')

NameError: name 'InstructorEmbeddings' is not defined

In [ ]:
#check retrieval relvance

query = 'Lebron James and Bronny James play in the same game'

retriever = index.as_retriever()

retriever.retrieve(query, k=5)
